# Apr ??, 2022 ()

**Motivation**:  <br>

In [ ]:
# HIDE CODE


import os
import sys
import nibabel as nib
import networkx as nx
from time import time
from pprint import pprint
from copy import deepcopy as dc
from os.path import join as pjoin
from numpy.ma import masked_where as mwh
from scipy.ndimage import gaussian_filter
from IPython.display import display, IFrame, HTML
from matplotlib.colors import rgb2hex, to_rgb
import matplotlib.pyplot as plt
import seaborn as sns

# set style & no interpolalation
import matplotlib
matplotlib.rcParams['image.interpolation'] = 'none'
sns.set_style('whitegrid')
%matplotlib inline

# tmp & extras dir
git_dir = pjoin(os.environ['HOME'], 'Dropbox/git')
tmp_dir = pjoin(git_dir, 'jb-Ca-fMRI/tmp')
extras_dir = pjoin(git_dir, 'jb-Ca-fMRI/_extras')
lfr_dir = pjoin(os.environ['HOME'], 'Documents/workspaces/lfr/binary_overlapping')


# GitHub
sys.path.insert(0, pjoin(git_dir, '_Ca-fMRI'))
from register.atlas import make_tree_graph
from register.parcellation import Parcellation
from analysis.hierarchical import Hierarchical
from analysis.fourier import *
from analysis.bootstrap import *
from analysis.svinet import *
from analysis.group import *
from analysis.lfr import *
from utils.render import *
from utils.plotting import *
from model.mouse import Mice
from model.configuration import Config

# warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Check Ca new

In [ ]:
rest_normal, led_normal = [], []
for i in range(1, 10 + 1):
    for j in range(1, 3 +1):
        for k in [2, 4, 6]:
            led_normal.append((i, j, k))
        for k in [1, 3, 5, 7]:
            rest_normal.append((i, j, k))
len(rest_normal), len(led_normal), len(rest_normal) + len(led_normal)

In [ ]:
pth = '/mnt/storage/hadi/Ca-fMRI/PreprocessedDataBIDS'

rest, led = [], []
for p in pathlib.Path(pth).rglob('*.nii.gz'):
    x = p.name.split('_')
    try:
        task = next(e for e in x if 'task' in e)
        task = task.split('-')[-1]
        
        sub = next(e for e in x if 'sub' in e)
        ses = next(e for e in x if 'ses' in e)
        run = next(e for e in x if 'run' in e)
        sub, ses, run = map(
            lambda s: int(re.findall(r"\d+", s).pop()),
            [sub, ses, run],
        )
        
        if task == 'led':
            led.append((sub, ses, run))
        elif task == 'rest':
            rest.append((sub, ses, run))
        else:
            print('???')
    except StopIteration:
        continue
        
led, rest = sorted(led), sorted(rest)
len(rest), len(led), len(rest) + len(led)

In [ ]:
led_delta = sorted(set(led_normal).symmetric_difference(led))
rest_delta = sorted(set(rest_normal).symmetric_difference(rest))

rest_delta, led_delta

In [ ]:
missing = [e for e in rest_delta if e[0] != 3]
missing += [(3, 1, 1), (3, 2, 1), (3, 2, 6), (6, 3, 2)]
missing.pop(missing.index((2, 2, 7)))
missing = sorted(missing)
missing

### Plot GS

In [ ]:
set_style()

In [ ]:
pth = '/mnt/storage/hadi/Ca-fMRI/PreprocessedDataBIDS'
paths = []
for p in pathlib.Path(pth).rglob('*.nii.gz'):
    if 'run' in p.name:
        paths.append(str(p))
paths = sorted(paths)

In [ ]:
q = 0.05
tstart = 1000

for p in paths:
    x = ants.image_read(p)
    x = x.numpy().squeeze()
    m = np.abs(x).mean(-1) > 0

    mu = x[..., tstart:].mean(-1)
    sd = x[..., tstart:].std(-1)
    tsnr = np.abs(mu) / sd
    thres = np.quantile(tsnr[m], q)
    
    gs_full = x[m].mean(0)
    gs_good = x[tsnr > thres].mean(0)
    gs_bad = x[tsnr <= thres].mean(0)
    
    # Plot
    mu, sd, tsnr, m = filter_boundaries([mu, sd, tsnr, m], m)

    fig = plt.figure(figsize=(13, 4.5), tight_layout=True)
    gspec = gridspec.GridSpec(2, 7, height_ratios=[1.2, 1])

    ax = fig.add_subplot(gspec[0, :])
    ax.plot(gs_full, color='yellow', alpha=0.7, lw=0.8)
    ax.plot(gs_good, color='k', alpha=0.8, lw=0.8, label='GS good')
    ax.plot(gs_bad, color='r',  alpha=0.5, lw=0.8, label='GS bad')
    ax.legend()
    ax.set_title(p.split('/')[-1])
    ax.grid()

    data2plt = {
        'mu': mu,
        'mu_hist': mu[m],
        'sd': sd,
        'sd_hist': sd[m],
        'tSNR': tsnr,
        'tSNR_hist': tsnr[m],
        'good_pix': tsnr > thres,
    }
    for i, (lbl, x2p) in enumerate(data2plt.items()):
        ax = fig.add_subplot(gspec[1, i])
        if 'hist' in lbl:
            sns.histplot(x2p, label=lbl.split('_')[0], ax=ax)
            ax.set_ylabel('')
            ax.set_yticks([])
            if 'tSNR' in lbl:
                ax.axvline(thres, color='r', lw='1.2', ls='--', label=f"{thres:0.2f}")
            ax.legend()
        else:
            ax.imshow(x2p, cmap='Spectral' if i == 4 else 'rocket')
            remove_ticks(ax)
    plt.show()
    

    sos = sp_sig.butter(2, 0.001, 'hp', fs=10, output='sos')
    y = sp_sig.sosfilt(sos, x)

    mu = y[..., tstart:].mean(-1)
    sd = y[..., tstart:].std(-1)
    tsnr = np.abs(mu) / sd

    m = np.abs(y).mean(-1) > 0
    thres = np.quantile(tsnr[m], q)

    gs_full = y[m].mean(0)
    gs_good = y[tsnr > thres].mean(0)
    gs_bad = y[tsnr <= thres].mean(0)

    # Plot
    mu, sd, tsnr, m = filter_boundaries([mu, sd, tsnr, m], m)

    fig = plt.figure(figsize=(13, 4.5), tight_layout=True)
    gspec = gridspec.GridSpec(2, 7, height_ratios=[1.2, 1])

    ax = fig.add_subplot(gspec[0, :])
    ax.plot(gs_full, color='yellow', alpha=0.7, lw=0.8)
    ax.plot(gs_good, color='k', alpha=0.8, lw=0.8, label='GS good')
    ax.plot(gs_bad, color='r',  alpha=0.5, lw=0.8, label='GS bad')
    ax.legend()
    ax.set_title(f"{p.split('/')[-1]}  . . .  Highpass filtered (0.001 Hz)")
    ax.grid()

    data2plt = {
        'mu': mu,
        'mu_hist': mu[m],
        'sd': sd,
        'sd_hist': sd[m],
        'tSNR': tsnr,
        'tSNR_hist': tsnr[m],
        'good_pix': tsnr > thres,
    }
    for i, (lbl, x2p) in enumerate(data2plt.items()):
        ax = fig.add_subplot(gspec[1, i])
        if 'hist' in lbl:
            sns.histplot(x2p, label=lbl.split('_')[0], ax=ax)
            ax.set_ylabel('')
            ax.set_yticks([])
            if 'tSNR' in lbl:
                ax.axvline(thres, color='r', lw='1.2', ls='--', label=f"{thres:0.2f}")
            ax.legend()
        else:
            ax.imshow(x2p, cmap='Spectral' if i == 4 else 'rocket')
            remove_ticks(ax)
    plt.show()
    
    print('\n\n')
    print('-' * 70)
    print('-' * 70)

In [ ]:
plt.imshow(mu)

In [ ]:
np.quantile(mu[m], 0.90)

In [ ]:
plt.imshow(m * (mu < np.quantile(mu[m], 0.95)))

In [ ]:
sns.histplot(mu[m])
plt.axvline(np.quantile(mu[m], 0.95), color='r')

## BOLD (Gabe)

In [ ]:
rest_normal, led_normal = [], []
for i in range(1, 10 + 1):
    for j in range(1, 3 +1):
        for k in [2, 4, 6]:
            led_normal.append((i, j, k))
        for k in [1, 3, 5, 7]:
            rest_normal.append((i, j, k))
len(rest_normal), len(led_normal), len(rest_normal) + len(led_normal)

### Clean

In [ ]:
pth = '/mnt/storage/hadi/Ca-fMRI/gabe_data/reoriented_data/cleaned_timeseries'

rest, led = [], []
for p in pathlib.Path(pth).rglob('*.nii.gz'):
    x = p.name.split('_')
    try:
        task = next(e for e in x if 'task' in e)
        task = task.split('-')[-1]
        
        sub = next(e for e in x if 'sub' in e)
        ses = next(e for e in x if 'ses' in e)
        run = next(e for e in x if 'run' in e)
        sub, ses, run = map(
            lambda s: int(re.findall(r"\d+", s).pop()),
            [sub, ses, run],
        )
        
        if task == 'led':
            led.append((sub, ses, run))
        elif task == 'rest':
            rest.append((sub, ses, run))
        else:
            print('???')
    except StopIteration:
        continue
        
led, rest = sorted(led), sorted(rest)
len(rest), len(led), len(rest) + len(led)

In [ ]:
len(rest_normal), len(led_normal), len(rest_normal) + len(led_normal)

In [ ]:
led_delta = sorted(set(led_normal).symmetric_difference(led))
rest_delta = sorted(set(rest_normal).symmetric_difference(rest))

rest_delta, led_delta

In [ ]:
missing = [e for e in rest_delta if e[0] != 3]
missing += [(1, 1, 4), (3, 1, 1)] #, (3, 2, 1), (3, 2, 6), (6, 3, 2)]
missing.pop(missing.index((2, 2, 7)))
missing = sorted(missing)
missing

### Raw

In [ ]:
pth = '/mnt/storage/hadi/Ca-fMRI/gabe_data/reoriented_data/native_preprocessed_data'

rest, led = [], []
for p in pathlib.Path(pth).rglob('*.nii.gz'):
    x = p.name.split('_')
    try:
        task = next(e for e in x if 'task' in e)
        task = task.split('-')[-1]
        
        sub = next(e for e in x if 'sub' in e)
        ses = next(e for e in x if 'ses' in e)
        run = next(e for e in x if 'run' in e)
        sub, ses, run = map(
            lambda s: int(re.findall(r"\d+", s).pop()),
            [sub, ses, run],
        )
        
        if task == 'led':
            led.append((sub, ses, run))
        elif task == 'rest':
            rest.append((sub, ses, run))
        else:
            print('???')
    except StopIteration:
        continue
        
led, rest = sorted(led), sorted(rest)
len(rest), len(led), len(rest) + len(led)

In [ ]:
led_delta = sorted(set(led_normal).symmetric_difference(led))
rest_delta = sorted(set(rest_normal).symmetric_difference(rest))

rest_delta, led_delta

In [ ]:
missing = [e for e in rest_delta if e[0] != 3]
# missing += [(1, 1, 4), (3, 1, 1)] #, (3, 2, 1), (3, 2, 6), (6, 3, 2)]
missing.pop(missing.index((2, 2, 7)))
missing = sorted(missing)
missing

### tSNR

In [ ]:
set_style()

In [ ]:
pth = '/mnt/storage/hadi/Ca-fMRI/gabe_data/reoriented_data/native_preprocessed_data'
pth_masks = '/mnt/storage/hadi/Ca-fMRI/gabe_data/reoriented_data/brain_mask'

paths = []
for p in pathlib.Path(pth).rglob('*.nii.gz'):
    if 'run' in p.name:
        paths.append(str(p))
paths = sorted(paths)

In [ ]:
slices = [20, 13, 38]
slices_delta = [15, 10, -12]
q = 0.1

for p in paths:
    x = ants.image_read(p)
    x = x.numpy().squeeze()
    
    m = '_'.join(p.split('/')[-1].split('_')[:4])
    m = next(filter(lambda f: m in f, os.listdir(pth_masks)))
    m = pjoin(pth_masks, m)
    m = ants.image_read(m)
    m = m.numpy().astype(bool)
    
    mu = x.mean(-1) * m
    sd = x.std(-1) * m
    tsnr = np.abs(mu) / sd
    thres = np.quantile(tsnr[m], q)
    
    gs_full = x[m].mean(0)
    gs_good = x[tsnr > thres].mean(0)
    gs_bad = x[tsnr <= thres].mean(0)
    
    # PLOT
    mu, sd, tsnr, m = filter_boundaries([mu, sd, tsnr, m], m)
    
    fig = plt.figure(figsize=(13, 8), tight_layout=True)
    gspec = GridSpec(4, 7, height_ratios=[1.3] + [1] * 3, width_ratios=[1] * 6 + [1.0])

    ax = fig.add_subplot(gspec[0, :])
    ax.plot(gs_full, color='magenta', alpha=1, lw=3, label='GS full')
    ax.plot(gs_good, color='k', alpha=1, lw=3, label='GS good')
    ax.plot(gs_bad, color='r',  alpha=1, lw=3, label='GS bad')
    ax.legend()
    ax.set_title(p.split('/')[-1])
    ax.grid()

    good_pix = tsnr > thres

    data2plt = {
        'mu': mu,
        'sd': sd,
        'tSNR': tsnr,
    }
    for ii, (lbl, xx) in enumerate(data2plt.items()):
        ax = fig.add_subplot(gspec[ii + 1, -1])
        sns.histplot(xx[m].ravel(), label=lbl, ax=ax)
        ax.set_ylabel('')
        ax.set_yticks([])

        if 'tSNR' in lbl:
            ax.axvline(thres, color='r', lw='1.2', ls='--', label=f"{thres:0.2f}")

        ax.legend()

        for jj, sl in enumerate(slices):
            ax = fig.add_subplot(gspec[ii + 1, 2 * jj])
            x2p = xx.take(sl, axis=jj)
            m2p = m.take(sl, axis=jj)
            x2p, m2p = filter_boundaries([x2p, m2p], m2p)
            if jj == 2:
                x2p = x2p.T
            ax.imshow(x2p, cmap='Spectral' if lbl == 'tSNR' else 'rocket')
            remove_ticks(ax)

            ax = fig.add_subplot(gspec[ii + 1, 2 * jj + 1])
            x2p = xx.take(sl + slices_delta[jj], axis=jj)
            m2p = m.take(sl + slices_delta[jj], axis=jj)
            x2p, m2p = filter_boundaries([x2p, m2p], m2p)
            if jj == 2:
                x2p = x2p.T
            ax.imshow(x2p, cmap='Spectral' if lbl == 'tSNR' else 'rocket')
            remove_ticks(ax)
    plt.show()
    
    print('\n\n')
    print('-' * 70)
    print('-' * 70)

### Power spectrum

In [ ]:
x = ants.image_read(p)
x = x.numpy().squeeze()

m = '_'.join(p.split('/')[-1].split('_')[:4])
m = next(filter(lambda f: m in f, os.listdir(pth_masks)))
m = pjoin(pth_masks, m)
m = ants.image_read(m)
m = m.numpy().astype(bool)

mu = x.mean(-1) * m
sd = x.std(-1) * m
tsnr = np.abs(mu) / sd
thres = np.quantile(tsnr[m], q)

In [ ]:
f, Pxx_den = sp_sig.periodogram(x, 1)

In [ ]:
plt.semilogy(f[1:], Pxx_den[tsnr > thres].mean(0)[1:])
# plt.ylim([1e-7, 1e2])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

In [ ]:
fig, axes = create_figure(1, 2, (8, 4), sharey='row')
axes[0].semilogy(f[1:], Pxx_den[tsnr > thres].mean(0)[1:])
axes[0].set_title('good vox')
axes[1].semilogy(f[1:], Pxx_den[tsnr <= thres].mean(0)[1:])
axes[1].set_title('bad vox')
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

### Missing?

In [ ]:
pth = '/mnt/storage/hadi/Ca-fMRI/gabe_data/reoriented_data/cleaned_timeseries'
paths = []
for p in pathlib.Path(pth).rglob('*.nii.gz'):
    if 'run' in p.name:
        paths.append(str(p))
paths = sorted(paths)

In [ ]:
p = paths[103]
p

In [ ]:
x = ants.image_read(p)
x = x.numpy().squeeze()

m = '_'.join(p.split('/')[-1].split('_')[:4])
m = next(filter(lambda f: m in f, os.listdir(pth_masks)))
m = pjoin(pth_masks, m)
m = ants.image_read(m)
m = m.numpy().astype(bool)

mu = x.mean(-1) * m
sd = x.std(-1) * m
tsnr = np.abs(mu) / sd
thres = np.quantile(tsnr[m], q)

gs_full = x[m].mean(0)
gs_good = x[tsnr > thres].mean(0)
gs_bad = x[tsnr <= thres].mean(0)

In [ ]:
# PLOT
mu, sd, tsnr, m = filter_boundaries([mu, sd, tsnr, m], m)

fig = plt.figure(figsize=(13, 8), tight_layout=True)
gspec = GridSpec(4, 7, height_ratios=[1.3] + [1] * 3, width_ratios=[1] * 6 + [1.0])

ax = fig.add_subplot(gspec[0, :])
ax.plot(gs_full, color='magenta', alpha=1, lw=3, label='GS full')
ax.plot(gs_good, color='k', alpha=1, lw=3, label='GS good')
ax.plot(gs_bad, color='r',  alpha=1, lw=3, label='GS bad')
ax.legend()
ax.set_title(p.split('/')[-1])
ax.grid()

good_pix = tsnr > thres

data2plt = {
    'mu': mu,
    'sd': sd,
    'tSNR': tsnr,
}
for ii, (lbl, xx) in enumerate(data2plt.items()):
    ax = fig.add_subplot(gspec[ii + 1, -1])
    sns.histplot(xx[m].ravel(), label=lbl, ax=ax)
    ax.set_ylabel('')
    ax.set_yticks([])

    if 'tSNR' in lbl:
        ax.axvline(thres, color='r', lw='1.2', ls='--', label=f"{thres:0.2f}")

    ax.legend()

    for jj, sl in enumerate(slices):
        ax = fig.add_subplot(gspec[ii + 1, 2 * jj])
        x2p = xx.take(sl, axis=jj)
        m2p = m.take(sl, axis=jj)
        x2p, m2p = filter_boundaries([x2p, m2p], m2p)
        if jj == 2:
            x2p = x2p.T
        ax.imshow(x2p, cmap='Spectral' if lbl == 'tSNR' else 'rocket')
        remove_ticks(ax)

        ax = fig.add_subplot(gspec[ii + 1, 2 * jj + 1])
        x2p = xx.take(sl + slices_delta[jj], axis=jj)
        m2p = m.take(sl + slices_delta[jj], axis=jj)
        x2p, m2p = filter_boundaries([x2p, m2p], m2p)
        if jj == 2:
            x2p = x2p.T
        ax.imshow(x2p, cmap='Spectral' if lbl == 'tSNR' else 'rocket')
        remove_ticks(ax)
plt.show()

In [ ]:
x = ants.image_read(p)
x = x.numpy().squeeze()

m = '_'.join(p.split('/')[-1].split('_')[:4])
m = next(filter(lambda f: m in f, os.listdir(pth_masks)))
m = pjoin(pth_masks, m)
m = ants.image_read(m)
m = m.numpy().astype(bool)

mu = x.mean(-1) * m
sd = x.std(-1) * m

In [ ]:
thres = np.quantile(sd[m], 1 - q)
thres

In [ ]:
f, Pxx_den = sp_sig.periodogram(x, 1)

In [ ]:
fig, axes = create_figure(1, 2, (8, 4), sharey='row')
axes[0].semilogy(f[1:], Pxx_den[sd < thres].mean(0)[1:])
axes[0].set_title('good vox')
axes[1].semilogy(f[1:], Pxx_den[sd >= thres].mean(0)[1:])
axes[1].set_title('bad vox')
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

In [ ]:
x2p = mwh(sd > thres, sd)

In [ ]:
plt.imshow(x2p[..., 20].T)